# Oppgave 4
Her skal du implementere et Pythonprogram (med bruk av SQL) som tar inn
en dato og skriver ut hvilke forestillinger som finnes på denne datoen og lister
opp hvor mange billetter (dvs. stoler) som er solgt. Ta også med forestillinger
hvor det ikke er solgt noen billetter.

In [5]:
import sqlite3
import pandas as pd
from tabulate import tabulate
from utils import get_file_name_of_newest_database

name = get_file_name_of_newest_database()
conn = sqlite3.connect(name)
cursor = conn.cursor()

In [6]:
def hent_forestilling_med_dato(date):
    # get forestilling with date and number of tickets sold
    query = """
    SELECT 
    F.TeaterStykkeNavn,
    F.Nummer as ForestillingNummer,
    F.Dato,
    F.Tid,
    count(B.ForestillingNummer) as AntallBilletter
    FROM Forestilling F
    LEFT JOIN Billett B
    ON F.Nummer = B.ForestillingNummer
    AND F.TeaterStykkeNavn = B.TeaterStykkeNavn
    Group BY F.TeaterStykkeNavn, F.Nummer
    HAVING F.Dato = ?
    """
    
    cursor.execute(query, (date,))
    # cursor.execute(query)
    forestillinger = cursor.fetchall()
    headers = [description[0] for description in cursor.description]
    return forestillinger, headers



# get all distinct forestillings_dato
query = """
SELECT DISTINCT Dato
FROM Forestilling
"""
cursor.execute(query)
dates = cursor.fetchall()

dates_string = "\n".join([f"{i+1}. {date[0]}" for i, date in enumerate(dates)])
selected_date = input(f'Velg en dato \n {dates_string}')
if selected_date.isdigit() and int(selected_date) <= len(dates) and int(selected_date) > 0:
    selected_date = dates[int(selected_date)-1][0]
else:
    print("Invalid input")
    exit()



forestilling_data, headers = hent_forestilling_med_dato(selected_date)
print(tabulate(forestilling_data, headers=headers , tablefmt="grid"))

+-------------------------------+----------------------+------------+-------+-------------------+
| TeaterStykkeNavn              |   ForestillingNummer | Dato       | Tid   |   AntallBilletter |
+===============================+======================+============+=======+===================+
| Størst av alt er kjærligheten |                    1 | 2024-02-03 | 18:30 |                36 |
+-------------------------------+----------------------+------------+-------+-------------------+
| Kongsemnene                   |                    3 | 2024-02-03 | 19:00 |                65 |
+-------------------------------+----------------------+------------+-------+-------------------+
